# Analysis


In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import MultiPoint, Point, Polygon,shape
import random
import warnings
import statsmodels.api as sm
from patsy import dmatrices
import geopy.distance
import pyperclip

In [114]:
labels={'acqua': 'Water fountains',
        'biblio':   'Public libraries',
        'ciclabili':'Bike lanes',
        'consu':    'Family counselling',
        'cult':	    'Cultural POI',
        'distr':    'Stores and Groceries',
        'edicole':  'News-stands',
        'farmacie':	'Pharmacies',
        'metro':    'Metro stations',
        'parchi':	'Public parks',
        'posta':    'Postal offices',
        'serd': 	'Addiction counselling',
        'sinf':	    'Kindergarten',
        'sita':	    'Italian schools',
        'sport':	'Sport facilities',
        'sprim':	'Elementary schools',
        'ss2':      'High schools',
        'ssec':     'Middle schools',
        'treni':    'Train stations',
        'uni':      'University buildings',
        'df_name':  'Service',
        'all': 'All services'
        }

col_list = ['acqua', 'biblio', 'ciclabili','consu','cult', 'edicole', 'farmacie', 'metro', 'parchi', 'posta', 'serd', 'sinf', 'sita', 'sport', 'sprim', 'ss2', 'ssec', 'treni', 'uni', 'wifi', 'distr','all']

## Data Cleaning and preprocessing

In [6]:
calc_df_fixed=pd.read_csv('../outputs/calc_fixed_df.csv', low_memory=False)
mm_df=pd.read_csv('../outputs/mm_dataset.csv')
#removing all non geolocated services
calc_df_fixed = calc_df_fixed.drop(calc_df_fixed[calc_df_fixed['long'].isna()].index)
calc_df_fixed

,name,long,lat,df_name,cat1,cat2,cat1_name,cat2_name,10001708,10001709,...,51039101,51039401,51039402,51039403,51039404,51039405,51039406,51039407,51039408,51039410
0,FORZE ARMATE,9.107364,45.456290,acqua,NaN,NaN,NaN,NaN,7540.1,7540.1,...,4609.7,8467.6,8467.6,8467.6,8467.6,8467.6,8467.6,8467.6,8467.6,8467.6
1,LORENTEGGIO,9.125151,45.458188,acqua,NaN,NaN,NaN,NaN,6445.5,6445.5,...,3735.9,7593.8,7593.8,7593.8,7593.8,7593.8,7593.8,7593.8,7593.8,7593.8
2,BANDE NERE,9.136149,45.461792,acqua,NaN,NaN,NaN,NaN,5797.8,5797.8,...,3165.9,7023.8,7023.8,7023.8,7023.8,7023.8,7023.8,7023.8,7023.8,7023.8
3,GIAMBELLINO,9.153322,45.452348,acqua,NaN,NaN,NaN,NaN,4673.6,4673.6,...,3846.8,7651.5,7651.5,7651.5,7651.5,7651.5,7651.5,7651.5,7651.5,7651.5
4,GRATOSOGLIO - Q.RE MISSAGLIA - Q.RE TERRAZZE,9.171006,45.408128,acqua,NaN,NaN,NaN,NaN,5655.9,5655.9,...,8094.3,10803.3,10803.3,10803.3,10803.3,10803.3,10803.3,10803.3,10803.3,10803.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13264,licenza madre ce palizzi,9.138032,45.507393,distr,alimentare|non alimentare,NaN,settore_merceologico,NaN,7640.8,7640.8,...,1647.7,3452.3,3452.3,3452.3,3452.3,3452.3,3452.3,3452.3,3452.3,3452.3
13265,NaN,9.097103,45.450728,distr,alimentare|non alimentare,NaN,settore_merceologico,NaN,8231.1,8231.1,...,5498.1,9356.0,9356.0,9356.0,9356.0,9356.0,9356.0,9356.0,9356.0,9356.0
13266,la rinascente s.p.a.,9.151735,45.489729,distr,alimentare|non alimentare,NaN,settore_merceologico,NaN,5841.5,5841.5,...,1041.9,4315.4,4315.4,4315.4,4315.4,4315.4,4315.4,4315.4,4315.4,4315.4
13269,NaN,9.158760,45.479925,distr,Non Alimentare|Alimentare|,NaN,settore_merceologico,NaN,5143.7,5143.7,...,1948.4,5203.1,5203.1,5203.1,5203.1,5203.1,5203.1,5203.1,5203.1,5203.1


Cleaning all housing units outside of comune di milano

In [ ]:
multipol = fiona.open(r"../datasets/milan_shape/L090102_ComuneMilano.shp")
multi = next(iter(multipol))

found=False
while not found:
    a=random.uniform(8.8, 9.37)
    b=random.uniform(45.35, 45.9)
    point = Point(a, b)
    found=point.within(shape(multi['geometry']))

print(a, b)

multipol = fiona.open(r"C:\Users\Jordi\Downloads\ESP_adm_shp\ESP_adm0.shp")
multi = next(iter(multipol))

point = Point(0,42)
point.within(shape(multi['geometry']))

## Summary statistics

In [ ]:
desc_mm=calc_df_fixed.groupby('df_name').describe()
desc_mm=desc_mm.drop(['long', 'lat'], axis=1)
desc_mm=desc_mm.T
desc_mm.head()

In [ ]:
min_services=calc_df_fixed.groupby('df_name').min()
min_services=min_services.drop(['long', 'lat', 'cat1_name', 'cat2_name'], axis=1)
desc_services=pd.DataFrame()
desc_services['min']=min_services.min(axis=1)
desc_services['id_min']=min_services.idxmin(axis=1)
max_services=calc_df_fixed.groupby('df_name').min()
max_services=max_services.drop(['long', 'lat', 'cat1_name', 'cat2_name'], axis=1)
desc_services['max']=max_services.max(axis=1)
desc_services['id_max']=max_services.idxmax(axis=1)
avg_services=calc_df_fixed.groupby('df_name').mean()
desc_services['avg']=avg_services.mean(axis=1)
sd_services=calc_df_fixed.groupby('df_name').std()
desc_services['sd']=avg_services.std(axis=1)
desc_services

In [ ]:
max_20=calc_df_fixed.groupby('df_name').min()
max_20=max_20.drop(['long', 'lat', 'cat1_name', 'cat2_name'], axis=1)
max_20=max_20.T
max_20

In [ ]:
plt.hist(max_20['ss2'])

# Index calculator

We construct the index as illustrated in the paper, note that in this notebook we just prove the concept, the full calculation takes place in another file (index.py)

In [71]:
def penality_function(x, dhat):
    if x <= dhat:
        return 1
    if x > dhat:
        return 1/(x-dhat+1)

def con_index(dist_list, dhat, n_max, alpha, beta):
    if  alpha + beta != 1:
        raise Exception("Alpha and Beta should sum to one")
    d_min = min(dist_list)
    close_list=[d for d in dist_list if d <=dhat]
    n = len(close_list)
    CI = alpha*penality_function(d_min, dhat)+beta*(n/n_max)
    return CI

con_index([9,6,6,6,6], 5, 3, 0.5, 0.5)

0.25

In [57]:
def best_in_class(df_name, dhat, df=calc_df_fixed):
    e_list=list(df.columns)
    e_list=[item for item in e_list if item not in ['name', 'long', 'lat','df_name', 'cat1', 'cat2', 'cat1_name', 'cat2_name']]
    nmax_dict=dict.fromkeys(e_list)
    if df_name=='all':
        pass
    else:
        df=df[df.df_name==df_name]
    for elem in e_list:
        dist_list=list(df[str(elem)])
        close_list=[d for d in dist_list if d <=dhat]
        nmax_dict[elem]=len(close_list)
    return max(nmax_dict.values())

best_in_class('biblio', 60*15)

3

In [ ]:
min(calc_df_fixed['10002602'].to_list())

In [97]:
mm_columns_list=['CODICE EDIFICIO', 'full_address', 'lat', 'lon']
index_df=mm_df[mm_columns_list]
index_df

,CODICE EDIFICIO,full_address,lat,lon
0,10001708,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
1,10001709,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
2,10001710,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
3,10001711,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
4,10001712,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053
...,...,...,...,...
975,51039405,VIA SENIGALLIA 60/G MILANO,45.532143,9.168970
976,51039406,VIA SENIGALLIA 60 MILANO,45.532143,9.168970
977,51039407,VIA SENIGALLIA 60 MILANO,45.532143,9.168970
978,51039408,VIA SENIGALLIA 60 MILANO,45.532143,9.168970


In [112]:
columns_list=list(calc_df_fixed.df_name.unique())+['all']
[best_in_class(name, 60*15)  for name in columns_list] #checking if function works for all dataframes

NameError: name 'best_in_class' is not defined

In [86]:
def fun_run(codice_edificio, dhat, alpha, beta, out_df, df=calc_df_fixed):
    columns_list=list(df.df_name.unique())+['all']
    #out_dict
    out_list=[]
    col_out_list=[]
    for column in columns_list:
        if column=='all':
            df_subset=df

        else:
            df_subset=df[df.df_name==column]
        dist_list=df_subset[str(codice_edificio)].to_list()
        if dist_list==[]:
            out=np.nan
            warnings.warn("Empty list")
        else:
            n_max=best_in_class(column, dhat)
            try:
                out=con_index(dist_list=dist_list, dhat=dhat, n_max=n_max, alpha=alpha, beta=beta)
            except:
                out=np.nan
                warnings.warn("Index not calculated because of some issue")
        col_out_list.append(column)
        out_list.append(out)
    if col_out_list == ['acqua', 'biblio', 'ciclabili','consu','cult', 'edicole', 'farmacie', 'metro', 'parchi', 'posta', 'serd', 'sinf', 'sita', 'sport', 'sprim', 'ss2', 'ssec', 'treni', 'uni', 'wifi', 'distr','all']:
        return out_list
    else:
        raise Exception("Not all columns are outputs of fun run")


fun_run(10003003, dhat=60*15, alpha=0.5, beta=0.5, out_df=index_df)

[1.0,
 0.8333333333333333,
 0.5570175438596491,
 0.8,
 0.5106382978723404,
 0.6120689655172413,
 0.7580645161290323,
 0.6,
 0.5982658959537572,
 0.6875,
 0.6,
 0.8928571428571428,
 0.5357142857142857,
 0.7586206896551724,
 0.85625,
 0.5833333333333334,
 0.686046511627907,
 0.0027716186252771603,
 0.5263157894736842,
 0.5943396226415094,
 0.547085201793722,
 0.6635044642857143]

## Index Analysis

We now perform the analysis of the index created

In [32]:
index_df=pd.read_csv('../outputs/index_df.csv')
print('Buildings that were not geocoded',len(index_df[index_df['lon'].isna()]))
index_df=index_df.drop(index_df[index_df['lon'].isna()].index) #dropping mm buildings that were not geocoded
index_df.loc[:, index_df.columns != 'list'].to_stata('../stata/index_df.dta') #exporting to dta file
index_df

Buildings that were not geocoded 49


C:\Users\iodio\AppData\Local\Temp\ipykernel_14276\2933445220.py:4: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    CODICE EDIFICIO   ->   CODICE_EDIFICIO

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  index_df.loc[:, index_df.columns != 'list'].to_stata('../stata/index_df.dta') #exporting to dta file


,CODICE EDIFICIO,full_address,lat,lon,list,acqua,biblio,ciclabili,consu,cult,...,sita,sport,sprim,ss2,ssec,treni,uni,wifi,distr,all
0,10001708,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
1,10001709,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
2,10001710,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
3,10001711,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
4,10001712,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,51039405,VIA SENIGALLIA 60/G MILANO,45.532143,9.168970,"[0.6666666666666666, 0.009293680297397777, 0.5...",0.666667,0.009294,0.527778,0.600000,0.000144,...,0.571429,0.594828,0.70000,0.002798,0.639535,0.625,0.000293,0.528302,0.502242,0.561384
976,51039406,VIA SENIGALLIA 60 MILANO,45.532143,9.168970,"[0.6666666666666666, 0.009293680297397777, 0.5...",0.666667,0.009294,0.527778,0.600000,0.000144,...,0.571429,0.594828,0.70000,0.002798,0.639535,0.625,0.000293,0.528302,0.502242,0.561384
977,51039407,VIA SENIGALLIA 60 MILANO,45.532143,9.168970,"[0.6666666666666666, 0.009293680297397777, 0.5...",0.666667,0.009294,0.527778,0.600000,0.000144,...,0.571429,0.594828,0.70000,0.002798,0.639535,0.625,0.000293,0.528302,0.502242,0.561384
978,51039408,VIA SENIGALLIA 60 MILANO,45.532143,9.168970,"[0.6666666666666666, 0.009293680297397777, 0.5...",0.666667,0.009294,0.527778,0.600000,0.000144,...,0.571429,0.594828,0.70000,0.002798,0.639535,0.625,0.000293,0.528302,0.502242,0.561384


In [29]:
def duomo_distance(lat, lon):
    coords=(lat, lon)
    duomo=(45.463968, 9.190578)
    return geopy.distance.geodesic(duomo, coords).m

In [34]:
index_df['duomo_dist']=index_df.apply(lambda x: duomo_distance(x['lat'], x['lon']), axis=1)

In [98]:
corr_df=index_df.corr()[3:].duomo_dist.reset_index()
corr_df.columns=['Service', 'Correlation']
corr_df=corr_df.replace({"Service": labels})
corr_df

C:\Users\iodio\AppData\Local\Temp\ipykernel_14276\655922510.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_df=index_df.corr()[3:].duomo_dist.reset_index()


,Service,Correlation
0,Water fountains,-0.367010
1,Public libraries,-0.368220
2,Bike lanes,-0.701029
3,Family counselling,-0.372839
4,Cultural POI,-0.294903
5,News-stands,-0.745758
6,Pharmacies,-0.724199
7,Metro stations,-0.405716
8,Public parks,-0.522855
9,Postal offices,-0.422085


In [99]:
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

In [102]:
corr_df['Mean'] = [index_df[column].mean() for column in list(index_df.loc[:, 'acqua':].columns)]
corr_df['Median'] = [index_df[column].median() for column in list(index_df.loc[:, 'acqua':].columns)]
corr_df['Minimum'] = [index_df[column].min() for column in list(index_df.loc[:, 'acqua':].columns)]
corr_df['Maximum'] = [index_df[column].max() for column in list(index_df.loc[:, 'acqua':].columns)]
corr_df['Standard deviation'] = [index_df[column].std() for column in list(index_df.loc[:, 'acqua':].columns)]
corr_df['Gini']=[gini(index_df[column]) for column in list(index_df.loc[:,'acqua':].columns)]
column_to_move = corr_df.pop("Correlation")
corr_df.insert(7, "Correlation", column_to_move)
pyperclip.copy(corr_df.to_latex(index=False))
corr_df

,Service,Mean,Median,Minimum,Maximum,Standard deviation,Gini,Correlation
0,Water fountains,0.536418,0.666667,0.000038,1.000000,0.331048,0.308046,-0.367010
1,Public libraries,0.408964,0.666667,0.000035,1.000000,0.340612,0.424246,-0.368220
2,Bike lanes,0.531777,0.577485,0.000043,1.000000,0.190943,0.154563,-0.701029
3,Family counselling,0.164648,0.000767,0.000032,1.000000,0.277174,0.746149,-0.372839
4,Cultural POI,0.052840,0.000391,0.000033,1.000000,0.175577,0.918119,-0.294903
5,News-stands,0.527958,0.560345,0.000039,1.000000,0.188203,0.152383,-0.745758
6,Pharmacies,0.559450,0.596774,0.000040,1.000000,0.212132,0.172591,-0.724199
7,Metro stations,0.350690,0.550000,0.000043,1.000000,0.311462,0.467732,-0.405716
8,Public parks,0.457354,0.523121,0.000036,1.000000,0.224840,0.229666,-0.522855
9,Postal offices,0.449311,0.562500,0.000039,1.000000,0.284404,0.319898,-0.422085


In [116]:
def return_min_max(df, columns_to_sort, id_to_return,id_name='ID', n=10):
    df_list=[]
    for column in columns_to_sort:
        df_max=df.sort_values(column, ascending=False).head(n).reset_index()
        df_max=df_max[[column, id_to_return]]
        df_max.columns=['Max', id_name+' max']
        df_min=df.sort_values(column, ascending=True).head(n).reset_index()
        df_min=df_min[[column, id_to_return]]
        df_min.columns=['Min', id_name+' min']
        df_min_max=pd.concat([df_max, df_min], axis=1)
        df_min_max.columns=pd.MultiIndex.from_product([[column], df_min_max.columns])
        df_list.append(df_min_max)
    return pd.concat(df_list, axis=1)
return_min_max(index_df, columns_list, 'CODICE EDIFICIO')

acqua                                  biblio                                \
    Max    ID max       Min    ID min       Max    ID max       Min    ID min   
0   1.0  51014001  0.000038  10004632  1.000000  41019801  0.000035  10004632   
1   1.0  20003010  0.000038  10003803  1.000000  10001801  0.000036  10003803   
2   1.0  51017301  0.000046  30001801  1.000000  41019802  0.000042  30001801   
3   1.0  20003001  0.000047  30002301  0.833333  10004616  0.000043  30002301   
4   1.0  20003002  0.000055  51014402  0.833333  10003006  0.000048  20004076   
5   1.0  20003003  0.000055  51014401  0.833333  10003005  0.000048  20004065   
6   1.0  20003004  0.000055  51014403  0.833333  10003004  0.000048  20004077   
7   1.0  20003005  0.000055  51014404  0.833333  10003003  0.000048  20004068   
8   1.0  20003006  0.000062  20004076  0.833333  10003002  0.000048  20004050   
9   1.0  20003007  0.000062  20004065  0.833333  10003001  0.000048  20004069   

  ciclabili            ...      wifi               distr                      \
        Max    ID max  ...       Min    ID min       Max    ID max       Min   
0  1.000000  51013801  ...  0.000039  10004632  1.000000  10004301  0.000038   
1  0.957602  10003802  ...  0.000039  10003803  0.986547  10004201  0.000039   
2  0.938596  30000801  ...  0.000048  30001801  0.919283  20000401  0.000047   
3  0.935673  10003801  ...  0.000048  30002301  0.883408  30000601  0.000047   
4  0.918129  30003402  ...  0.000054  51014404  0.831839  30003701  0.000055   
5  0.910819  51011801  ...  0.000054  51014402  0.820628  20001001  0.000055   
6  0.906433  30003401  ...  0.000054  51014401  0.784753  30000501  0.000055   
7  0.887427  30000301  ...  0.000054  51014403  0.748879  30000502  0.000055   
8  0.887427  10004401  ...  0.000061  41039806  0.748879  30000504  0.000057   
9  0.862573  30000701  ...  0.000061  41039803  0.733184  30000503  0.000057   

                  all                                
     ID min       Max    ID max       Min    ID min  
0  10004632  1.000000  10004301  0.000043  10004632  
1  10003803  0.980469  20000401  0.000043  10003803  
2  30001801  0.974330  10004201  0.000054  30001801  
3  30002301  0.952567  20000201  0.000054  30002301  
4  51014401  0.938058  20001001  0.000065  20004076  
5  51014404  0.921875  20000101  0.000065  20004065  
6  51014403  0.920201  20000103  0.000066  20004077  
7  51014402  0.914062  20000102  0.000066  20004068  
8  20004076  0.904576  30000601  0.000066  20004050  
9  20004065  0.901786  51013801  0.000066  51014404  

[10 rows x 88 columns]

In [35]:
y, X = dmatrices('duomo_dist ~  acqua + biblio + ciclabili+ consu + cult + edicole + farmacie + metro + parchi + posta + serd + sinf + sita + sport + sprim + ss2 + ssec + treni + uni + wifi + distr + all', data=index_df, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             duomo_dist   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     72.73
Date:                Thu, 06 Jul 2023   Prob (F-statistic):          6.21e-183
Time:                        11:51:21   Log-Likelihood:                -8481.1
No. Observations:                 931   AIC:                         1.701e+04
Df Residuals:                     908   BIC:                         1.712e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.418e+04    236.716     59.919      0.0